# Session, Context and Environment concepts

Vortex initialisation:

In [1]:
%load_ext ivortex
%vortex tmpcocoon

# [2019/09/03-17:19:30][vortex.sessions][_set_rundir:0152][WARNING]: Session <root> is changing its working directory </sauvegarde/vortex-dev/vortex/examples/notebooks/tutorial>
# [2019/09/03-17:19:30][vortex.sessions][_set_rundir:0155][INFO]: Session <root> set rundir </home/meunierlf/vortex-workdir/auto_cocoon_r8bzh14b>


Vortex 1.6.2 loaded ( Tuesday 03. September 2019, at 17:19:29 )
The working directory is now: /home/meunierlf/vortex-workdir/auto_cocoon_r8bzh14b/root


'/home/meunierlf/vortex-workdir/auto_cocoon_r8bzh14b'

## Principle

At some point, you have to be able to: 

  * Access global configuration data 
  * Call system functions (changing directory, copying files, ...) 
  * Have the list of files that have been retrieved/sent 
  * Modify environment variables, go back, ... 

This is translated into Vortex by a tree of objects: 

* One or more *Session(s)* each described by a session *Ticket* object. A session *Ticket* is made of: 

  * A *Glove* object (GLObal Versatile Environment) that contains global configuration information (user name, user profile, ...) 
  * A *System* object that will give access to a large number of system functions. 
  * A *Context* object whose members are: 
      
      - A *Sequence* object that preserves the memory of the different sections that were created during the data retrieval and archiving phases
      - One (or more) *Environment* object(s) that allows to interact with environment variables.
      - Possible sub-contexts 

In the description above, we notice that there can be several *Sessions*, *Contexts* or *Environments* objects. However, at a given time, only one of these objects will be “active” and the hierarchical link will be enforced (the active *Context* is necessarily a member of the active *Session*). 

**Example :** to have several sub-*Context*s can be interesting if, in the same script, we need to carry out two distinct tasks.

  * We will then create two sub-contexts (*Ctx1* and *Ctx2*);
  * We activate *Ctx1* and we start to perform the first task in a subdirectory of its own: retrieval of resources, changes to the environment, etc;
  * At some point, we can decide to start the second task. To do so, we activate *Ctx2* in order to jump into a dedicated directory and start from a clean situation (i.e. the environment of *Ctx2* does not contain anymore the modifications made in *Ctx1* when carrying out the previous task);
  * We can very well decide to go back to work on the first task... It is then enough to activate again *Ctx1* so that the environment of the first task is restored. 
  * ... 

## In practice...

The user or task developer should not worry too much about creating *Sessions*, *Contexts*... because:

  * By default, *Session* and *Context* objects are created when loading Vortex: for many scripts this is enough because only one task is carried out at a time (this is particularly the case with Olive).
  * In the case of a multi-tasking job, this *Sessions* and *Contexts* creation work is done by the “official” job creation system (presentation of the last day).

It should be remembered that, at some point, there is an active *Session*, *Context* and *Environment*. The rest of the presentation will focus on the recovery of these “active” objects and their use.

Retrieval of the active session (or more precisely of its *Ticket*): 

In [2]:
t = vortex.ticket()
# the session, in itself, is not very interesting but it gives access 
# to many features 

###  The *Glove* of the active session

In [3]:
print(t.glove.idcard())

+ User     = meunierlf
+ Profile  = research
+ Vapp     = play
+ Vconf    = sandbox
+ Configrc = /home/meunierlf/.vortexrc


In [4]:
# One of the few points to customise in the glove is the default vapp/vconf
t.glove.vapp = 'arpege'
t.glove.vconf = '4dvarfr'
# (this is pre-configured by Olive or bu the job creation system)

### The *System* object of the active session

In [5]:
print(t.sh)

<vortex.tools.systems.Linux34p object at 0x7f05100a9400 | footprint=7>


This object is of a very common use, we will return to it later in the presentation. 

### The active *Context* object

In [6]:
print(t.context)

Each *Context* has a path to identify it in the hierarchy. Here we work on the _“``root``”_ session (created by default when launching Vortex) and on the _“``root``”_ context (created automatically during the creation of the session): 

In [7]:
print(t.context.path)

/root/root


### The active context's *Sequence* and *Environment* objects

In [8]:
print(t.context.sequence)

In [9]:
print(t.context.env)

<vortex.tools.env.Environment object at 0x7f0510084be0 | including 105 variables>


These objects are of a very common use, we will come back to them later in the  of the presentation. 

## The *System*: access to the features of the OS

The *System* object provides access to many methods that make it easy to access operating system features. 

The *System* object: 

  * Defines a number of Vortex-specific methods (cp, rm, touch, spawn, ...) 
  * If a method is not redefined in the *System* object, it gives transparent access to the methods and modules made available by Python in the standard ``os``, ``shutil`` and ``resource`` modules. 

In [10]:
# Name of the current directory 
current = t.sh.pwd()
# Change the current directory
print(t.sh.cd('/tmp'))
print(t.sh.pwd())
# and come back...
print(t.sh.cd(current))

True
/tmp
True


In [11]:
files_list = t.sh.ls()

print(files_list)

['']


The methods of this object are very numerous, so we are not going to describe all of them here. For a detailed description, see the Sphinx documentation for the **vortex.tools.systems module**:

http://intra.cnrm.meteo.fr/algopy/sphinx/vortex/current/library/vortex/tools/systems.html 

As mentioned before, the *System* object gives access to methods or modules available in standard Python modules such as ``os``, ``shutil`` and ``resource`` . For example, in this way, we often use the **os.path** Python's module: 

In [12]:
print('Exists ?', t.sh.path.exists('/dev/null'))
print('Basename:', t.sh.path.basename('/dev/null'))
print('Dirname:', t.sh.path.dirname('/dev/null'))

Exists ? True
Basename: null
Dirname: /dev


For documentation on these methods and modules, refer to the official Python documentation.

### Type of *System* objects

System objects are not necessarily the same from one machine to another. In this notebook, we have:

In [13]:
type(t.sh)

vortex.tools.systems.Linux34p

The class of the *System* object is chosen when loading Vortex and creating the default session. This choice is made via the **footprints** package, based on the data returned by the Python **platform** module (type of architecture, operating system, Python version ...). 

Very often the ``Linux27`` or ``Linux34p`` objects are chosen but there are also variants for MacOS or formerly for NEC-SX. Having different classes depending on the versions allows different implementations depending on the operationg system type, which is very important for long-term maintenance. Indeed, a number of functions of the **os** Python module clearly shows that their implementations are not portable from one architecture to another, so we have to reserve us the right to create specific implementations.

From a practical point of view, *System* objects for the different architectures all inherit from the **vortex.tools.systems.OSExtended** class, which can be seen as an abstract class: all methods in this class can be used regardless of the operating system.

**Requirement in the development of Vortex**: the *System* object must always be used for any interaction with the operating system, even if in the case of ``t.sh.path``, we _currently_ rely on a standard implementation. 

### Shell extension (addons)

You can add new features to the *System* object on the fly. Such *Addons* can be used to add methods that are dedicated to a given model or data format (these are optional things).

The easiest way is to illustrate their use by an example: 

In [14]:
class HelloAddon(vortex.tools.addons.Addon):  # This class should be defined in the Vortex source tree
    
    _footprint = dict(
        attr = dict(
            kind = dict(values = ['hello']),
        )
    )
    
    def say_hi(self):
        self.sh.title("Hello World !")

shhello = fp.proxy.addon(kind='hello', sh=sh)  # Loading the HelloAddon (at the beginning of the script)

In [15]:
t.sh.say_hi()  # Voila !


=                                          HELLO WORLD !                                           =



**Principle**: The public methods of the *Addons* object (*i.e.* those whose name does not begin with ``_``) are directly accessible from the *System* object. 

### Alter behavior according to the data format

For some data formats, standard methods should not be used. For example, Arpege now uses splitted FA/LFIs (in-house format) which are actually directories containing multiple files glued together by an index file; for these data:

  * a simple ``cp`` or ``mv`` does not work (you have to call a specific command);
  * the ``diff`` does not work because the fields are not written in a deterministic order;
  * you have to concatenate the different files of the directory before archiving them (specific program).

To satisfy this type of needs, it is possible to customise certain methods depending on the format. These are the methods: ``cp``, ``mv``, ``rm``, ``diff``, ``ftget``, ``rawftget``, ``ftput``, ``rawftput``, ``scpput`` and ``scpget``.

When an (optional) attribute ``fmt='format'`` is specified when calling one of these methods (for example, ``cp``), the *System* object checks if it has a method named ``format_cp``. In such a case, it is called instead of the standard ``cp`` function.

Objects calling these methods should therefore be careful to provide valid ``fmt`` attributes (otherwise the “standard” method is used). In Vortex, the object resource's handler always provides a format to *Store* objects (that are in charge of data handling).

In practice, methods associated with specific formats are defined in *Addons* so that users who do not need them are not “polluted”: 

In [16]:
print('Before:', hasattr(t.sh, 'lfi_cp'))
# the t.sh.cp('origin', 'destination', fmt="lfi") command will perform a standard cp :-(
import vortex.tools.lfi
shlfi = fp.proxy.addon(kind='lfi', sh=t.sh)
print('After:', hasattr(t.sh, 'lfi_cp'))
# the t.sh.cp('origin', 'destination', fmt="lfi") command will now do the right thing.

Before: False
After: True


## The active context's *Sequence* object

First, let's get some data for the demonstration to make sense.

In [17]:
r1 = toolbox.input(role='ArpegeNamelist', now=True, fatal=False, verbose=False, loglevel='warn',
                   # Resource
                   kind='namelist', model='arpege', source='something',
                   # Provider
                   genv='uget:cy42_op2.01fake@ugetdemo',
                   # Container
                   local='namelvoid')
r2 = toolbox.input(role='ArpegeNamelist', now=True, verbose=False, loglevel='warning',
                   # Resource
                   kind='namelist', model='arpege', source='namelistfc',
                   # Provider
                   genv='uget:cy42_op2.01fake@ugetdemo',
                   # Container
                   local='nameltest')
r2 = toolbox.input(role='RawGridpoint', now=True, verbose=False, loglevel='warning',
                   # Container
                   local='epytest_[geometry:area]_[term]', format='grib',
                   # Resource
                   kind='gridpoint', geometry='antil0025', origin='historic',
                   date='2016100100', term='0,3', cutoff='production', model='[vapp]',
                   nativefmt='[format]',
                   # Provider
                   experiment='OPER', block='forecast', namespace='vortex.multi.fr',
                   vapp='arome', vconf='antilles')

# [2019/09/03-17:19:32][vortex.tools.systems][cp:2211][ERROR]: Missing source /home/meunierlf/vortex-workdir/auto_cocoon_r8bzh14b/root/cy42_op2.09lf.nam/something
# [2019/09/03-17:19:32][vortex.tools.systems][cp:2211][ERROR]: Missing source /home/meunierlf/vortex-workdir/auto_cocoon_r8bzh14b/root/cy42_op2.09lf.nam/something
# [2019/09/03-17:19:32][vortex.data.stores][_verbose_log:0528][WARNING]: Multistore get uget://uget.multi.fr: none of the opened store succeeded.


In [18]:
seq = t.context.sequence

Retrieve the list of previously created *Sections* (*Warning:* we are actualy talking about *Sections*, not resource's *Handler*s): 

In [19]:
seq.inputs()

<generator object Sequence.inputs at 0x7f04f77b0e60>

In [20]:
print('\n'.join(['{0.rh.container.abspath:s} ({0.role:s}, stage={0.stage})'.format(section)
                 for section in seq.inputs()]))

/home/meunierlf/vortex-workdir/auto_cocoon_r8bzh14b/root/namelvoid (ArpegeNamelist, stage=void)
/home/meunierlf/vortex-workdir/auto_cocoon_r8bzh14b/root/nameltest (ArpegeNamelist, stage=get)
/home/meunierlf/vortex-workdir/auto_cocoon_r8bzh14b/root/epytest_ANTIL0025_00:00 (RawGridpoint, stage=get)
/home/meunierlf/vortex-workdir/auto_cocoon_r8bzh14b/root/epytest_ANTIL0025_03:00 (RawGridpoint, stage=get)


The **_``stage``_** attribute allows us to know the state of the resource: 

  * *load*: the section has been created but nothing else has been done (``now=False``);
  * *checked*: the remote resource exists (*check* method) but has not been retrieved; 
  * *void*: error, the requested resource does not exist; 
  * *get*: the resource has been retrieved. 

In general, we want to get the list of *Sections*:
    • For which the *get* method was successfuly called (stage=get) ;
    • For a specific ``role`` (or for a given ``kind``).

The **effective_inputs** method is provided for this purpose:

In [21]:
print('\n'.join(['{0.rh.container.abspath:s} ({0.role:s}, stage={0.stage})'.format(section)
                 for section in seq.effective_inputs(role='RawGridpoint')]))

/home/meunierlf/vortex-workdir/auto_cocoon_r8bzh14b/root/epytest_ANTIL0025_00:00 (RawGridpoint, stage=get)
/home/meunierlf/vortex-workdir/auto_cocoon_r8bzh14b/root/epytest_ANTIL0025_03:00 (RawGridpoint, stage=get)


In [22]:
print('\n'.join(['{0.rh.container.abspath:s} ({0.role:s}, stage={0.stage})'.format(section)
                 for section in seq.effective_inputs(role='ArpegeNamelist')]))
#NB: The “void” section does not appear

/home/meunierlf/vortex-workdir/auto_cocoon_r8bzh14b/root/nameltest (ArpegeNamelist, stage=get)


In [23]:
# It is possible to use regular expressions by compiling them before use...
import re
regex = re.compile('(Arpege|Arome|Aladin)Namelist')
print('\n'.join(['{0.rh.container.abspath:s} ({0.role:s}, stage={0.stage})'.format(section)
                 for section in seq.effective_inputs(role=regex)]))

/home/meunierlf/vortex-workdir/auto_cocoon_r8bzh14b/root/nameltest (ArpegeNamelist, stage=get)


We can also rely on the ``kind`` attribute of the resource (to avoid as much as possible):

In [24]:
print('\n'.join(['{0.rh.container.abspath:s} ({0.role:s}, stage={0.stage})'.format(section)
                 for section in seq.effective_inputs(kind='namelist')]))

/home/meunierlf/vortex-workdir/auto_cocoon_r8bzh14b/root/nameltest (ArpegeNamelist, stage=get)


**Note:** In the *Sequence* object, equivalent methods exist for the output data (**outputs** and **effective_outputs**) but are much less used... 

## The active context's *Environment* object

Using the current context, we can retrieve the active *Environment* object at any time: 

In [25]:
e = t.context.env
print(e)

<vortex.tools.env.Environment object at 0x7f0510084be0 | including 105 variables>


It is also possible to use the following shortcut:

In [26]:
e = t.env
print(e)

<vortex.tools.env.Environment object at 0x7f0510084be0 | including 105 variables>


As a first approach, we can consider that the *Environment* object is used like a Python dictionary (with a few additions) ... 

Variables are accessible in several ways:

In [27]:
print(e['HOME'])
print(e.HOME)
print(e.home)

/home/meunierlf
/home/meunierlf
/home/meunierlf


Asking for a missing variable returns ``None``... 

In [28]:
print('FOO exists ?', 'FOO' in e)
print(e.FOO)

FOO exists ? False
None


There are some handy functions:

In [29]:
for var in [1, 'yes', 'ok', 'true']:
    e.FOO = var
    print("is True ?", e.true('FOO'))

is True ? True
is True ? True
is True ? True
is True ? True


Even more interesting, it is possible to make temporary changes:

In [30]:
e.FOO = 1
with e.delta_context(FOO=2, TOTO='is here'):
    print('foo={0.foo}, toto {0.toto}'.format(e))

# Changes are "forgotten" once you leave the context manager:
print('foo={0.foo}, toto {0.toto}'.format(e))

foo=2, toto is here
foo=1, toto None


In [31]:
del e.FOO

There are some additional methods. For those who are interested: 

* http://intra.cnrm.meteo.fr/algopy/sphinx/vortex/current/library/vortex/tools/env.html
* http://intra.cnrm.meteo.fr/algopy/sphinx/vortex/current/technical/environment.html (see the examples at the end of this document)

## Conclusion

These notions of *Session*, *Context*, *Sequence* and *Environment* can be scary, but for most users and developers, just knowing how to retrieve “active” objects (in order to use them) is enough. 

For those who would like to know much more about these objects:
http://intra.cnrm.meteo.fr/algopy/sphinx/vortex/current/technical/environment.html 

Any questions: *vortex.support@meteo.fr*